In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/計畫/ntut-project-in-os2d-channel-prune

In [ ]:
# !mkdir visualized_image

### Init

In [ ]:
# !ls

In [ ]:
# !pip install yacs

In [ ]:
import torch

def show_gpu_memory_usage():
    """顯示當前 GPU RAM 使用情況"""
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(device)

        # 獲取記憶體使用情況 (轉換為 GB)
        allocated = torch.cuda.memory_allocated(device) / 1024**3
        reserved = torch.cuda.memory_reserved(device) / 1024**3
        total = torch.cuda.get_device_properties(device).total_memory / 1024**3

        print(f"🖥️  GPU 設備: {device_name}")
        print(f"📊 記憶體使用情況:")
        print(f"   已分配: {allocated:.2f} GB")
        print(f"   已保留: {reserved:.2f} GB")
        print(f"   總容量: {total:.2f} GB")
        print(f"   使用率: {(allocated/total)*100:.1f}%")
        print(f"   保留率: {(reserved/total)*100:.1f}%")

        # 視覺化進度條
        usage_percent = int((allocated/total)*100)
        bar_length = 20
        filled_length = int(bar_length * usage_percent / 100)
        bar = '█' * filled_length + '░' * (bar_length - filled_length)
        print(f"   [{bar}] {usage_percent}%")

    else:
        print("❌ CUDA 不可用，無法檢測 GPU 記憶體")

In [ ]:
import os
import argparse

import torch
import os
import argparse
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

from os2d.modeling.model import build_os2d_from_config
from os2d.config import cfg
import  os2d.utils.visualization as visualizer
from os2d.structures.feature_map import FeatureMapSize
from os2d.utils import setup_logger, read_image, get_image_size_after_resize_preserving_aspect_ratio
from os2d.data import dataloader
from os2d.modeling.model import build_os2d_from_config

from os2d.data.dataloader import build_eval_dataloaders_from_cfg, build_train_dataloader_from_config
from os2d.engine.train import trainval_loop
from os2d.utils import set_random_seed, get_trainable_parameters, mkdir, save_config, setup_logger, get_data_path
from os2d.engine.optimization import create_optimizer
from os2d.config import cfg
from os2d.utils.visualization import *
import random
import os2d.utils.visualization as visualizer
from pathlib import Path
import cv2
import numpy as np
from os2d.utils import get_image_size_after_resize_preserving_aspect_ratio
from src.util.detection import generate_detection_boxes
from src.util.visualize import visualize_boxes_on_image
from src.util.filter import DataLoaderDB

In [ ]:
if cfg.is_cuda:
    assert torch.cuda.is_available(), "Do not have available GPU, but cfg.is_cuda == 1"
    torch.backends.cudnn.benchmark = True

# random seed
set_random_seed(cfg.random_seed, cfg.is_cuda)

# Model
net, box_coder, criterion, img_normalization, optimizer_state = build_os2d_from_config(cfg)

# Optimizer
parameters = get_trainable_parameters(net)
optimizer = create_optimizer(parameters, cfg.train.optim, optimizer_state)

# load the dataset
data_path = get_data_path()
dataloader_train, datasets_train_for_eval = build_train_dataloader_from_config(cfg, box_coder, img_normalization,
                                                                                data_path=data_path)

dataloaders_eval = build_eval_dataloaders_from_cfg(cfg, box_coder, img_normalization,
                                                    datasets_for_eval=datasets_train_for_eval,
                                                    data_path=data_path)

db = DataLoaderDB( path = './src/db/data.csv' , dataloader = dataloader_train)

### Test Basic Method of DB

In [ ]:
image_ids = list ( map( int , db.get_image_ids()) )
sorted_image_ids = sorted(image_ids)
print( sorted_image_ids )

### Test Basic Method of ContextAoiAlign

In [ ]:
from src.lcp.ct_aoi_align import ContextAoiAlign
transform_image = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(img_normalization["mean"], img_normalization["std"])
                      ])

context_aoi_align = ContextAoiAlign( db, dataloader_train, transform_image , net , cfg )

In [ ]:
from src.lcp.aux_net import AuxiliaryNetwork
aux_net = AuxiliaryNetwork( context_aoi_align, db )

#### Test for LCP

In [ ]:
from src.lcp.lcp import LCP

In [ ]:
show_gpu_memory_usage()

In [ ]:
lcp = LCP(net, aux_net, dataloader_train)
lcp.init_for_indices()

In [ ]:
show_gpu_memory_usage()

### Create DB for keep indices

In [ ]:
from src.util.prune_db import PruneDBControler
prune_db = PruneDBControler( path = './src/db/prune_channel_information.csv' )
# prune_db.initial()

In [ ]:

layers = lcp.get_layers_name()
for name, ch in layers:
    if name == 'layer2.0.conv2':
        pass
    else:
        continue
    print(f"{name}: {ch} channels")
    keep, discard = lcp.get_channel_selection_by_no_grad(
        layer_name   = f"net_feature_maps.{name}",
        discard_rate = 0.5,
        lambda_rate  = 1.0,
        use_image_num= 3,
        random_seed  = 42
    )
    print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")
    # prune_db.write_data(
    #     layer = f"net_feature_maps.{name}",
    #     original_channel_num= len(keep) + len(discard),
    #     num_of_keep_channel = len(keep),
    #     keep_index  = keep
    # )

In [ ]:
show_gpu_memory_usage()

### Test for Pruner

In [ ]:
from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )

In [ ]:
show_gpu_memory_usage()

In [ ]:
# dependencies = pruner.resolve_layer_dependencies('net_feature_maps.layer2.0.conv3')
# print("依賴關係分析結果:")
# print(f"目標層級: {dependencies['target']}")
# print(f"BatchNorm 層: {dependencies['batch_norm']}")
# print(f"下游層級: {dependencies['downstream_layers']}")
# print(f"跳躍連接: {dependencies['skip_connections']}")
# print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:
# dependencies = pruner.resolve_layer_dependencies('net_feature_maps.layer2.0.conv3')
# print("依賴關係分析結果:")
# print(f"目標層級: {dependencies['target']}")
# print(f"BatchNorm 層: {dependencies['batch_norm']}")
# print(f"下游層級: {dependencies['downstream_layers']}")
# print(f"跳躍連接: {dependencies['skip_connections']}")
# print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:
# for name, ch in layers:
#     dependencies = pruner.resolve_layer_dependencies(f'net_feature_maps.{name}')
#     print("依賴關係分析結果:")
#     print(f"目標層級: {dependencies['target']}")
#     print(f"BatchNorm 層: {dependencies['batch_norm']}")
#     print(f"下游層級: {dependencies['downstream_layers']}")
#     print(f"跳躍連接: {dependencies['skip_connections']}")
#     print(f"剪枝策略: {dependencies['pruning_strategy']}")

In [ ]:
# # 測試改進後的依賴分析
# layer_name = "net_feature_maps.layer3.0.conv3"
# dependencies = pruner.resolve_layer_dependencies(layer_name)

# print("依賴分析結果:")
# print(f"BatchNorm 層: {dependencies['batch_norm']}")
# print(f"下游層級: {dependencies['downstream_layers']}")
# print(f"Downsample 層: {dependencies['downsample_layer']}")
# print(f"跳躍連接: {dependencies['skip_connections']}")
# print(f"剪枝策略: {dependencies['pruning_strategy']}")

# # 執行剪枝
# if dependencies['downsample_layer']:
#     print(f"✅ 找到 downsample 層: {dependencies['downsample_layer']}")
# else:
#     print("ℹ️  該層沒有對應的 downsample 層")


In [ ]:
# # 測試參數設定
# layer_name = 'net_feature_maps.layer2.0.conv2'
# keep_indexes = prune_db.get_layer_keep_indices(layer_name)
# dependencies = pruner.resolve_layer_dependencies(layer_name)
# print(f"🔍 測試層級: {layer_name}")
# print(f"📊 保留通道索引: {keep_indexes}")
# print(f"📈 保留通道數: {len(keep_indexes)}")


In [ ]:
# print( dependencies )

In [ ]:
# print( pruner.prune_network)

In [ ]:
# # === 步驟 1: 輸出通道剪枝 ===
# print("\n=== 步驟 1: 輸出通道剪枝 ===")
# success = pruner._prune_out_channel(layer_name, keep_indexes)

# if success:
#     print("✅ 輸出通道剪枝成功")

#     # 驗證輸出通道剪枝結果
#     target_layer = pruner._get_layer_by_name(layer_name)
#     print(f"   剪枝後輸出通道數: {target_layer.out_channels}")
#     print(f"   權重形狀: {target_layer.weight.shape}")
# else:
#     print("❌ 輸出通道剪枝失敗")

# # === 步驟 2: BatchNorm 剪枝 ===
# print("\n=== 步驟 2: BatchNorm 剪枝 ===")
# bn_layer_name = dependencies.get('batch_norm')
# print(f"🎯 BatchNorm 層: {bn_layer_name}")

# if bn_layer_name:
#     success = pruner._prune_batchnorm_layer(bn_layer_name, keep_indexes)

#     if success:
#         print("✅ BatchNorm 剪枝成功")

#         # 驗證 BatchNorm 剪枝結果
#         bn_layer = pruner._get_layer_by_name(bn_layer_name)
#         print(f"   BatchNorm 特徵數: {bn_layer.num_features}")
#         print(f"   權重形狀: {bn_layer.weight.shape}")
#         print(f"   偏置形狀: {bn_layer.bias.shape}")
#     else:
#         print("❌ BatchNorm 剪枝失敗")
# else:
#     print("⚠️  未找到對應的 BatchNorm 層")

# # === 步驟 3: 下游層輸入通道剪枝 ===
# print("\n=== 步驟 3: 下游層輸入通道剪枝 ===")
# downstream_layers = dependencies.get('downstream_layers', [])

# if downstream_layers:
#     next_layer_name = downstream_layers[0]
#     print(f"🎯 下游層: {next_layer_name}")

#     # 檢查下游層剪枝前的狀態
#     next_layer = pruner._get_layer_by_name(next_layer_name)
#     if next_layer:
#         print(f"剪枝前下游層輸入通道數: {next_layer.in_channels}")
#         print(f"剪枝前下游層權重形狀: {next_layer.weight.shape}")

#         # 執行下游層輸入通道剪枝
#         success = pruner._prune_next_layer_inputs(layer_name, next_layer_name, keep_indexes)

#         if success:
#             print("✅ 下游層輸入通道剪枝成功")

#             # 驗證下游層剪枝結果
#             next_layer = pruner._get_layer_by_name(next_layer_name)
#             print(f"   剪枝後輸入通道數: {next_layer.in_channels}")
#             print(f"   剪枝後權重形狀: {next_layer.weight.shape}")

#             # 驗證維度一致性
#             if next_layer.in_channels == len(keep_indexes):
#                 print("   ✅ 維度一致性檢查通過")
#             else:
#                 print("   ❌ 維度一致性檢查失敗")
#         else:
#             print("❌ 下游層輸入通道剪枝失敗")
#     else:
#         print("❌ 無法獲取下游層對象")
# else:
#     print("⚠️  未找到下游層")

# # === 步驟 4: Downsample 剪枝 ===
# print("\n=== 步驟 4: Downsample 剪枝 ===")
# skip_connections = dependencies.get('skip_connections', [])

# if skip_connections:
#     print(f"🎯 跳躍連接: {skip_connections}")

#     # 檢查是否需要處理 downsample
#     downsample_processed = False

#     for skip_conn in skip_connections:
#         if 'downsample' in skip_conn:
#             print(f"🔧 處理 Downsample 層: {skip_conn}")

#             # 檢查 downsample 層是否存在
#             if pruner._verify_layer_exists(skip_conn):
#                 downsample_layer = pruner._get_layer_by_name(skip_conn)

#                 if downsample_layer:
#                     print(f"   Downsample 層類型: {type(downsample_layer).__name__}")

#                     # 檢查 downsample 剪枝前的狀態
#                     if hasattr(downsample_layer, '__iter__'):  # Sequential
#                         print("   Downsample 結構分析:")
#                         for i, sub_layer in enumerate(downsample_layer):
#                             print(f"     [{i}] {type(sub_layer).__name__}", end="")
#                             if hasattr(sub_layer, 'out_channels'):
#                                 print(f" - 輸出通道: {sub_layer.out_channels}")
#                             elif hasattr(sub_layer, 'num_features'):
#                                 print(f" - 特徵數: {sub_layer.num_features}")
#                             else:
#                                 print()

#                     # 執行 downsample 剪枝
#                     try:
#                         pruner._prune_downsample_connection(skip_conn, keep_indexes)
#                         print("   ✅ Downsample 剪枝成功")
#                         downsample_processed = True

#                         # 驗證 downsample 剪枝結果
#                         downsample_layer = pruner._get_layer_by_name(skip_conn)
#                         if hasattr(downsample_layer, '__iter__'):
#                             print("   剪枝後 Downsample 結構:")
#                             for i, sub_layer in enumerate(downsample_layer):
#                                 print(f"     [{i}] {type(sub_layer).__name__}", end="")
#                                 if hasattr(sub_layer, 'out_channels'):
#                                     print(f" - 輸出通道: {sub_layer.out_channels}")
#                                 elif hasattr(sub_layer, 'num_features'):
#                                     print(f" - 特徵數: {sub_layer.num_features}")
#                                 else:
#                                     print()

#                         # 驗證維度一致性
#                         conv_layer = downsample_layer[0] if hasattr(downsample_layer, '__iter__') else downsample_layer
#                         if hasattr(conv_layer, 'out_channels') and conv_layer.out_channels == len(keep_indexes):
#                             print("   ✅ Downsample 維度一致性檢查通過")
#                         else:
#                             print("   ❌ Downsample 維度一致性檢查失敗")
#                     except Exception as e:
#                         print(f"   ❌ Downsample 剪枝失敗: {e}")
#                 else:
#                     print(f"   ❌ 無法獲取 Downsample 層對象: {skip_conn}")
#             else:
#                 print(f"   ⚠️  Downsample 層不存在: {skip_conn}")

#         elif skip_conn == 'residual_addition':
#             print("   🔗 檢測到殘差連接，但無需額外處理")

#     if not downsample_processed:
#         print("   ℹ️  無需處理 Downsample 層")
# else:
#     print("⚠️  未檢測到跳躍連接")

# # === 步驟 5: 整體驗證 ===
# print("\n=== 步驟 5: 整體驗證 ===")

# # 驗證剪枝一致性
# print("🔍 剪枝一致性檢查:")

# # 1. 檢查目標層
# target_layer = pruner._get_layer_by_name(layer_name)
# if target_layer:
#     print(f"   目標層 {layer_name}: {target_layer.out_channels} 通道")

# # 2. 檢查 BatchNorm 層
# if bn_layer_name:
#     bn_layer = pruner._get_layer_by_name(bn_layer_name)
#     if bn_layer:
#         print(f"   BatchNorm {bn_layer_name}: {bn_layer.num_features} 特徵")
#         if bn_layer.num_features == target_layer.out_channels:
#             print("   ✅ 目標層與 BatchNorm 維度一致")
#         else:
#             print("   ❌ 目標層與 BatchNorm 維度不一致")

# # 3. 檢查下游層
# if downstream_layers:
#     next_layer = pruner._get_layer_by_name(downstream_layers[0])
#     if next_layer:
#         print(f"   下游層 {downstream_layers[0]}: {next_layer.in_channels} 輸入通道")
#         if next_layer.in_channels == target_layer.out_channels:
#             print("   ✅ 目標層與下游層維度一致")
#         else:
#             print("   ❌ 目標層與下游層維度不一致")

# # 4. 檢查 Downsample 層
# downsample_layers = [conn for conn in skip_connections if 'downsample' in conn]
# if downsample_layers:
#     for downsample_name in downsample_layers:
#         if pruner._verify_layer_exists(downsample_name):
#             downsample_layer = pruner._get_layer_by_name(downsample_name)
#             if downsample_layer and hasattr(downsample_layer, '__iter__'):
#                 conv_layer = downsample_layer[0]
#                 if hasattr(conv_layer, 'out_channels'):
#                     print(f"   Downsample {downsample_name}: {conv_layer.out_channels} 輸出通道")
#                     if conv_layer.out_channels == target_layer.out_channels:
#                         print("   ✅ 目標層與 Downsample 維度一致")
#                     else:
#                         print("   ❌ 目標層與 Downsample 維度不一致")


In [ ]:
# # === 步驟 7: 通道索引追蹤測試 ===
# print("\n=== 步驟 7: 通道索引追蹤測試 ===")

# print("🎯 執行通道索引追蹤...")

# try:
#     # 執行通道索引追蹤
#     pruner.track_channel_indices(layer_name, keep_indexes)
#     print("   ✅ 通道索引追蹤成功")

#     # 驗證追蹤結果
#     print("   🔍 追蹤結果驗證:")

#     # 1. 檢查剪枝數據庫是否更新
#     if hasattr(pruner, 'prune_db') and pruner.prune_db:
#         try:
#             stored_indices = pruner.prune_db.get_layer_keep_indices(layer_name)
#             if stored_indices:
#                 print(f"     - 數據庫中保存的索引數量: {len(stored_indices)}")
#                 print(f"     - 索引一致性檢查: {'✅ 一致' if stored_indices == keep_indexes else '❌ 不一致'}")
#             else:
#                 print("     - ⚠️  數據庫中未找到保存的索引")
#         except Exception as e:
#             print(f"     - ❌ 數據庫檢查失敗: {e}")

#     # 2. 檢查內部追蹤記錄
#     if hasattr(pruner, '_prune_history'):
#         if layer_name in pruner._prune_history:
#             history = pruner._prune_history[layer_name]
#             print(f"     - 原始通道數: {history.get('original_channels', 'N/A')}")
#             print(f"     - 保留通道數: {history.get('kept_channels', 'N/A')}")
#             print(f"     - 剪枝比例: {history.get('prune_ratio', 0):.2%}")
#         else:
#             print("     - ⚠️  內部追蹤記錄中未找到該層")

#     # 3. 計算剪枝統計
#     original_channels = target_layer.out_channels if target_layer else len(keep_indexes)
#     kept_channels = len(keep_indexes)
#     prune_ratio = 1.0 - (kept_channels / original_channels) if original_channels > 0 else 0.0

#     print(f"     - 實際剪枝統計:")
#     print(f"       * 保留通道: {kept_channels}")
#     print(f"       * 剪枝比例: {prune_ratio:.2%}")
#     print(f"       * 壓縮效果: {(1-prune_ratio)*100:.1f}% 保留")

# except Exception as e:
#     print(f"   ❌ 通道索引追蹤失敗: {e}")
#     import traceback
#     traceback.print_exc()


In [ ]:
# # === 步驟 8: 完整性驗證與測試報告 ===
# print("\n=== 步驟 8: 完整性驗證與測試報告 ===")

# # 網路前向傳播測試
# print("🧪 網路前向傳播測試:")
# try:
#     # 創建測試輸入
#     device = next(pruner.prune_network.parameters()).device
#     test_input = torch.randn(1, 3, 224, 224, device=device)

#     # 設置網路為評估模式
#     pruner.prune_network.eval()

#     with torch.no_grad():
#         # 嘗試前向傳播
#         output = pruner.prune_network.net_feature_maps(test_input)
#         print(f"   ✅ 前向傳播測試成功")
#         print(f"   📊 輸入形狀: {test_input.shape}")
#         print(f"   📊 輸出形狀: {output.shape}")

# except Exception as e:
#     print(f"   ❌ 前向傳播測試失敗: {e}")

# # 生成測試報告
# print("\n📋 === 完整測試報告 ===")

# test_results = {
#     '輸出通道剪枝': True,  # 根據實際結果設置
#     'BatchNorm 剪枝': True,
#     '下游層輸入剪枝': True,
#     'Downsample 剪枝': downsample_processed if 'downsample_processed' in locals() else False,
#     '輸入通道修正': True,  # 根據實際結果設置
#     '通道索引追蹤': True,
#     '前向傳播測試': True   # 根據實際結果設置
# }

# print(f"🎯 測試層級: {layer_name}")
# print(f"📊 保留通道數: {len(keep_indexes)}")
# print(f"📈 剪枝比例: {(1 - len(keep_indexes) / 512) * 100:.1f}%")  # 假設原始通道數為 512

# print("\n🔍 測試結果詳情:")
# for test_name, result in test_results.items():
#     status = "✅ 通過" if result else "❌ 失敗"
#     print(f"  {test_name}: {status}")

# overall_success = all(test_results.values())
# print(f"\n🏆 整體測試結果: {'🎉 全部通過！' if overall_success else '⚠️ 部分測試失敗'}")

# if overall_success:
#     print("\n✨ 恭喜！您的 LCP 通道剪枝功能運作正常")
#     print("🚀 包含 Track 和 Fix 功能的完整剪枝流程已驗證")
#     print("📝 建議進行更多層級的測試以確保穩定性")
# else:
#     failed_tests = [name for name, result in test_results.items() if not result]
#     print(f"\n🔧 需要檢查的項目: {', '.join(failed_tests)}")
#     print("💡 建議逐項檢查失敗的功能並進行修正")

# print(f"\n🎉 剪枝流程完成！保留 {len(keep_indexes)} 個通道")


In [ ]:
# pruner.print_detailed_layers()

In [ ]:
# print( pruner.prune_network )

In [ ]:
# # === 維度一致性完整驗證 ===
# print("\n=== 維度一致性完整驗證 ===")

# def verify_channel_consistency(pruner, current_layer, next_layer, keep_indexes):
#     """驗證通道維度一致性"""
#     try:
#         current = pruner._get_layer_by_name(current_layer)
#         next_conv = pruner._get_layer_by_name(next_layer)

#         print(f"當前層 {current_layer}:")
#         print(f"  輸出通道數: {current.out_channels}")
#         print(f"  保留通道數: {len(keep_indexes)}")

#         print(f"下游層 {next_layer}:")
#         print(f"  輸入通道數: {next_conv.in_channels}")

#         # 檢查一致性
#         if current.out_channels == len(keep_indexes) == next_conv.in_channels:
#             print("✅ 通道維度完全一致")
#             return True
#         else:
#             print("❌ 通道維度不一致")
#             return False

#     except Exception as e:
#         print(f"❌ 驗證過程發生錯誤: {e}")
#         return False

# # 執行驗證
# consistency_ok = verify_channel_consistency(
#     pruner, layer_name, next_layer_name, keep_indexes
# )


##### Test For Prune One Layer

In [ ]:
# # 測試參數設定
# layer_name = 'net_feature_maps.layer3.0.conv2'
# keep_indexes = prune_db.get_layer_keep_indices(layer_name)
# dependencies = pruner.resolve_layer_dependencies(layer_name)
# print(f"🔍 測試層級: {layer_name}")
# print(f"📊 保留通道索引: {keep_indexes}")
# print(f"📈 保留通道數: {len(keep_indexes)}")


In [ ]:
# pruner.prune_layer(
#     layer_name   = layer_name
# )

In [ ]:
# print( pruner.prune_network )

In [ ]:

# layers = lcp.get_layers_name()
# for name, ch in layers:
#     if name == 'layer3.0.conv3':
#         pass
#     else:
#         continue
#     print(f"{name}: {ch} channels")
#     keep, discard = lcp.get_channel_selection_by_no_grad(
#         layer_name   = f"net_feature_maps.{name}",
#         discard_rate = 0.5,
#         lambda_rate  = 1.0,
#         use_image_num= 3,
#         random_seed  = 42
#     )
#     print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")
#     prune_db.write_data(
#         layer = f"net_feature_maps.{name}",
#         original_channel_num= ch,
#         num_of_keep_channel = len(keep),
#         keep_index  = keep
#     )

### Test For Prune Layer in LCP

In [ ]:
print( layers )
layer_names = []
for name, ch in layers:
    if name.endswith('.conv1') or name.endswith('.conv2'):
        layer_names.append( name )

In [ ]:
lcp.set_prune_db(prune_db)

#### Before Prune

In [ ]:
show_gpu_memory_usage()

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 2
)

### 這裡要跑 62 minutes for ALL

In [ ]:
# for idx, layer_name in enumerate(layer_names):
#     if idx != 15:
#         continue
#     lcp.prune_layer(
#         layer_name   = layer_name,
#         discard_rate = 0.8,
#     )
#     break

#### Test for debug for vision after prune

In [ ]:
# lcp.debug_for_test_vision(
#     dataloader_train = dataloader_train,
#     img_normalization = img_normalization,
#     box_coder = box_coder,
#     cfg = cfg,
#     count = 1
# )

In [ ]:
import torch

def count_parameters(model):
    """
    計算模型的參數數量
    Returns:
      total_params: 包含所有參數
      trainable_params: 只包含 requires_grad=True 的參數
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

import io

def estimate_model_size(model):
    """
    將模型序列化到緩衝區，估算存檔大小（MB）
    """
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    size_mb = buffer.getbuffer().nbytes / (1024 ** 2)
    return size_mb



In [ ]:
# # 使用範例
# print( "原始網路參數統計:\n" )
# model = lcp._net  
# total, trainable = count_parameters(model)
# print(f"總參數量: {total:,}")
# print(f"可訓練參數量: {trainable:,}")

# size_mb = estimate_model_size(model)
# print(f"模型存儲大小: {size_mb:.2f} MB")

# print( "剪枝網路參數統計:\n" )
# model = lcp._prune_net
# total, trainable = count_parameters(model)
# print(f"總參數量: {total:,}")
# print(f"可訓練參數量: {trainable:,}")

# size_mb = estimate_model_size(model)
# print(f"模型存儲大小: {size_mb:.2f} MB")

### Test for finetune

In [ ]:
show_gpu_memory_usage()

In [ ]:
from src.lcp.lcpfinetune import LCPFineTune
lcp_finetune = LCPFineTune(
    prune_net = lcp._prune_net,
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg       = cfg,
    optimizer=optimizer,
    parameters=parameters
)

In [ ]:
show_gpu_memory_usage()

In [ ]:
lcp_finetune._setup_logging()

In [ ]:
cfg.defrost()

cfg.train.optim.max_iter = 100
cfg.train.do_training = True
cfg.output.print_iter = 10
cfg.eval.iter = 10
cfg.train.batch_size = 1

cfg.freeze()


In [ ]:
from src.util.loss import LCPFinetuneCriterion

In [ ]:

lcp_criterion = LCPFinetuneCriterion(
    original_criterion=criterion,  # 原始損失函數實例
    aux_net=lcp._aux_net,  # 包含 aux_loss 方法的實例
    auxiliary_weight=1.0  # 可以調整輔助損失權重
)

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 1
)

In [ ]:
def show_network_status( orig_net, prune_net ):
    print( "原始網路參數統計:\n" )
    model = orig_net
    total, trainable = count_parameters(model)
    print(f"總參數量: {total:,}")
    print(f"可訓練參數量: {trainable:,}")

    size_mb = estimate_model_size(model)
    print(f"模型存儲大小: {size_mb:.2f} MB")

    print( "剪枝網路參數統計:\n" )
    model = prune_net
    total, trainable = count_parameters(model)
    print(f"總參數量: {total:,}")
    print(f"可訓練參數量: {trainable:,}")

    size_mb = estimate_model_size(model)
    print(f"模型存儲大小: {size_mb:.2f} MB")

In [ ]:
for idx, layer_name in enumerate(layer_names):
    print(f"剪枝前的網路統計: {idx} {layer_name}")

In [ ]:
# for idx, layer_name in enumerate(layer_names):
#     if layer_name == 'layer3.0.conv1' or layer_name == 'layer3.1.conv1' or layer_name == 'layer3.2.conv1' or layer_name == 'layer3.3.conv1' or layer_name == 'layer3.4.conv1' or layer_name == 'layer3.5.conv1':
#         continue
#     cfg.defrost()
#     if idx > 13:
#         cfg.train.optim.max_iter = 50
#         lcp.prune_layer(
#             layer_name   = layer_name,
#             discard_rate = 0.5,
#         )
#     else:
#         cfg.train.optim.max_iter = 1
#         lcp.prune_layer(
#             layer_name   = layer_name,
#             discard_rate = 0.8,
#         )
#     cfg.freeze()
#     lcp_finetune.start_finetune(
#         criterion= lcp_criterion,  # 使用自定義損失函數
#     )
#     show_gpu_memory_usage()
#     print(f"剪枝後網路狀態 (第 {idx+1}, {layer_name} 層):")
#     show_network_status(lcp._net, lcp_finetune._prune_net)
#     lcp._prune_net = lcp_finetune._prune_net  # 更新剪枝網路
#     lcp.save_checkpoint_with_pruned_net(
#         log_path = './src/util/checkpoints-test',
#         optimizer = optimizer,
#         model_name = f"lcp_finetune_{idx+1}_{layer_name}",
#         i_iter = cfg.train.optim.max_iter
#     )
#     lcp.debug_for_test_vision(
#         dataloader_train = dataloader_train,
#         img_normalization = img_normalization,
#         box_coder = box_coder,
#         cfg = cfg,
#         count = 2
#     )
#     break

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 2
)

In [ ]:
# lcp_finetune.finetune_one_batch(
#     batch_data = data,
#     lcp_instance = lcp,
#     criterion = criterion
# )

In [ ]:
# lcp_finetune.finetune_loop(
#     lcp_instance = lcp,
#     criterion = criterion,
#     dataloaders_eval = dataloaders_eval,
#     max_iters = 5,
#     print_interval=2
# )

In [ ]:
show_network_status(lcp._net, lcp._prune_net)

In [ ]:
print( lcp._pruner.resolve_layer_dependencies( "net_feature_maps.layer1.1.conv1"))

In [ ]:
prune_db._get_all_data_by_layer( 'layer1.0.conv1')

In [ ]:
# lcp._prune_net = lcp._net

In [ ]:
# all_layers = lcp._prune_db.get_all_layers()
# pruned_layers = [layer for layer in all_layers if layer.startswith('layer')]
        
# print( pruned_layers )

In [ ]:
# pruned_net_checkpoint = lcp.load_checkpoint_with_pruned_net(
#     checkpoint_file_path = './src/util/checkpoints-test/checkpoint_lcp_finetune_1_layer1.0.conv1.pth'
# )

In [ ]:
print( lcp._prune_net)

In [ ]:
show_network_status(lcp._net, lcp._prune_net)

In [ ]:
# for i in range(250):
#     print(50 * '=' + f" {i+1} / 250 開始全局微調" + 50 * '=')
#     lcp_finetune.start_finetune(
#         criterion= lcp_criterion,  # 使用自定義損失函數
#     )
#     show_gpu_memory_usage()
#     show_network_status(lcp._net, lcp_finetune._prune_net)
#     lcp._prune_net = lcp_finetune._prune_net  # 更新剪枝網路
#     lcp.debug_for_test_vision(
#         dataloader_train = dataloader_train,
#         img_normalization = img_normalization,
#         box_coder = box_coder,
#         cfg = cfg,
#         count = 1
#     )

### Test for recontruction

In [ ]:
from src.lcp.recontruction import LCPReconstruction
lcp_reconstruction = LCPReconstruction(
    prune_db = prune_db,
    pruner = pruner,
    prune_net = lcp._prune_net
)

In [ ]:
lcp_reconstruction.load_checkpoint_with_pruned_net(
    './src/util/checkpoints-test/checkpoint_lcp_finetune_1_layer1.0.conv1.pth'
)

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 2
)

In [ ]:
lcp._prune_net = lcp_reconstruction._prune_net
show_network_status(lcp._net, lcp._prune_net)

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 2
)

In [ ]:
pruned_layers = []
for layer in lcp_reconstruction._pruned_layers:
    if layer not in pruned_layers:
        pruned_layers.append(layer)
print( pruned_layers)

In [ ]:
for layer in pruned_layers:
    lcp.prune_layer(
        layer_name   = layer,
        discard_rate = None,
    )

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 2
)

In [ ]:
show_network_status(lcp._net, lcp._prune_net)